In [39]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)
examples = [
    {
        "movie": "탑건",
        "answer": "🛩️👨‍✈️🔥"
    },
    {
        "movie": "대부",
        "answer": "👨‍👨‍👦🔫🍝"
    },
    {
        "movie": "인셉션",
        "answer": "💤🌀🏙️"
    },
    {
        "movie": "기생충",
        "answer": "🏠🐛💰"
    },
]

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]





def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{movie} 영화 제목을  가져와 영화를 나타내는 세개의 이모티콘으로 응답해줘."),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie expert, you give short answers."),
        example_prompt,
        ("human", "{movie}영화 제목을  가져와 영화를 나타내는 세개의 이모티콘으로 응답해줘."),
    ]
)
result = chain.invoke({"movie": "Interstellar"})
memory.save_context({"input": "Interstellar"}, {"output": result.content})

chain.invoke({"movie": "Interstellar"})

🚀🌌⏳🚀🌌⏳

AIMessageChunk(content='🚀🌌⏳')

In [40]:
result = chain.invoke({"movie": "대부"})
memory.save_context({"input": "대부"}, {"output": result.content})

👨‍👨‍👦🔫🍝

In [41]:
print(memory.load_memory_variables({})["history"])

[HumanMessage(content='Interstellar'), AIMessage(content='🚀🌌⏳'), HumanMessage(content='대부'), AIMessage(content='👨\u200d👨\u200d👦🔫🍝')]
